## Does `EnergyModel` actually work for estimation?

The notebook runs a load of automated tests and gets the `EnergyModel` estimation of energy usage, then compares it with the values from energy.json.

In [1]:
from env import TestEnv
import pandas as pd
import json
import os
from conf import LisaLogging
from bart.common.Utils import area_under_curve
from trappy.plotter import plot_trace
from IPython.display import display
from trappy import ILinePlot
from trappy.stats.grammar import Parser
import pandas as pd
from trace import Trace
LisaLogging.setup()
import logging
logging.getLogger('Trace').setLevel(logging.ERROR)
logging.getLogger('Analysis').setLevel(logging.WARNING)
%matplotlib inline
from platforms.juno_energy import juno_energy
from platforms.pixel_energy import pixel_energy

2017-01-18 16:04:00,474 INFO    : root         : Using LISA logging configuration:
2017-01-18 16:04:00,475 INFO    : root         :   /home/brendan/sources/lisa/logging.conf


In [2]:
import tests.eas.generic
from tests.eas.generic import EnergyModelTest

## Run all the tests
This will probalby take ages

In [3]:
# Disable drawing power from USB
# !adb -s GA0113TP0180 shell "echo 'echo 0 > /sys/class/power_supply/battery/charging_enabled' | su"

In [4]:
scs = EnergyModelTest.__subclasses__()

In [5]:
scs

[tests.eas.generic.OneSmallTask,
 tests.eas.generic.ThreeSmallTasks,
 tests.eas.generic.TwoBigTasks,
 tests.eas.generic.TwoBigThreeSmall,
 tests.eas.generic.RampUp,
 tests.eas.generic.RampDown,
 tests.eas.generic.EnergyModelWakeMigration]

In [8]:
tests = []
all_experiments = []
measured_energy = []
est_energy = []

for cls in [scs[-1]]:
    cls.setUpClass()
    if not cls.te.emeter:
        print "\nWARNING: no energy meter configured\n"
    t = cls()
    tests.append(t)
    for exp in t.executor.experiments:
        all_experiments.append(exp)
        json_path = os.path.join(exp.out_dir, "energy.json")
        if not os.path.exists(json_path):
            print "\nWARNING: no energy.json, stuff probably won't work"
            continue
        with open(json_path) as f:
            energy = json.load(f)
        energy = sum(energy.values())
        measured_energy.append(energy)
                
        est_power = t.get_power_df(t.executor.experiments[0])['power']
        est_energy.append(area_under_curve(est_power))

2017-01-18 16:04:24,025 INFO    : LisaTest     : Setup tests execution engine...
2017-01-18 16:04:24,027 INFO    : TestEnv      : Using base path: /home/brejac01/sources/lisa
2017-01-18 16:04:24,028 INFO    : TestEnv      : Loading default (file) target configuration
2017-01-18 16:04:24,029 INFO    : TestEnv      : Loading target configuration [/home/brejac01/sources/lisa/target.config]...
2017-01-18 16:04:24,035 INFO    : TestEnv      : Loading custom (inline) test configuration
2017-01-18 16:04:24,036 INFO    : TestEnv      : External tools using:
2017-01-18 16:04:24,038 INFO    : TestEnv      :    ANDROID_HOME: /work/android-sdk-linux
2017-01-18 16:04:24,039 INFO    : TestEnv      :    CATAPULT_HOME: /home/brendan/sources/lisa/tools/catapult
2017-01-18 16:04:24,040 INFO    : TestEnv      : Devlib modules to load: ['bl', 'cpufreq', 'cgroups', 'hwmon']
2017-01-18 16:04:24,042 INFO    : TestEnv      : Connecting Android target [HT6670300102]
2017-01-18 16:04:24,043 INFO    : TestEnv   

# Compare `EnergyModel` estimation with measured value

In [9]:
df = pd.DataFrame({'measured': measured_energy, 'estimated': est_energy}, columns=['measured', 'estimated'])
if not df.empty:
    df.plot.scatter(x='estimated', y='measured')

In [10]:
tests

[<tests.eas.generic.EnergyModelWakeMigration testMethod=runTest>]

## Show results

Now we'll plot:

- Task residency
- CPU frequency (if available)
- Energy recorded by energy meter (if available)
- Energy estimated:
  - By `sched_group_energy` in the kernel (using the `sched_energy_diff` trace event)
  - By `EnergyModel`, according to the ideal utilization values that would be expected for the observed task placement
  - By `EnergyMOdel`, according to the utilization values extracted via the `sched_load_avg_cpu` event

In [11]:
t = tests[0]
ex = t.executor.experiments[0]
trace = t.get_trace(ex)

Maximum estimated system energy: 5428


In [12]:
trace = Trace(t.te.platform, trace.data_dir, trace.available_events + ['cpu_util', 'walt_update_task_ravg'])

Maximum estimated system energy: 5428


In [13]:
util_df = trace.ftrace.cpu_util.data_frame.groupby(level=0).first().pivot(columns='cpu').util_avg.ffill()
# Fix my cockup with '==' instead of '=' in the trace_printk call
# util_df = util_df.apply(lambda row: [float((i or '=-1')[1:]) for i in row])

In [14]:
df = Parser(trace.ftrace).solve('sched_load_avg_cpu:util_avg').reindex(util_df.index, method='ffill').fillna(-1)
# df = pd.concat([df, util_df], axis=1, keys=['A', 'B'])
ILinePlot([df, util_df], column=[0], drawstyle='steps-post').view()

""
""


In [15]:
plot_trace(trace.ftrace)

In [16]:
em = t.te.nrg_model

In [17]:
est_idle_df = em.mimic_sched_group_energy(trace, component='idle')
est_active_df = em.mimic_sched_group_energy(trace, component='active')

In [18]:
def get_estimations_df(trace, component=None, column=None, nrg_model=pixel_energy):
    # Get energy estimated by scheduler's sched_group_energy (this is a trace event added by me)
    # To make this the most useful, hack the kernel so that it always computes energy for all sched_groups
    if 'sg_energy' in trace.available_events:
        df = trace.ftrace.sg_energy.data_frame
        df = df.groupby(level=0).last() # drop rows with duplicate index to placate `pivot`
        sched_est = df.pivot(columns='sg_cpus')[['idle', 'active']].fillna(method='ffill')
        if component is None:
            sched_est = sched_est['idle'] + sched_est['active']
        else:    
            sched_est = sched_est[component]

        if column is None:
            sched_est = sched_est.sum(axis=1)
        else:
            cpus_string = hex(reduce(lambda x, y: x | (1 << y), column, 0))
            sched_est = sched_est[cpus_string]
    else:
        print "No sg_energy event, won't get sched_group_energy data from lunix kernal"
        sched_est = pd.DataFrame(columns=['sg_energy'])                
        
    # Get detailed estimation according to EnergyModel
    def get_model_est(model):
        si = None
        if not sched_est.empty:
            si = sched_est.index
        model_est = model.mimic_sched_group_energy(trace, component=component, sample_index=si,
                                                  flags='use_power_domains')
        if column is None:
            model_est = model_est.sum(axis=1)
        else:
            if column in model_est:
                model_est = model_est[column]
            else:
                model_est = pd.DataFrame(columns=[column])
        return model_est
        
    model_est = get_model_est(nrg_model)
    flat_model_est = get_model_est(nrg_model.flatten_energy())
    flat_pd_model_est = get_model_est(nrg_model.flatten_power_domains())
        
    # model_est = model_est.reindex(sched_est.index, method='ffill')
    
    df = pd.concat([sched_est, model_est, flat_model_est, flat_pd_model_est], axis=1).ffill()

    #df = df.sort_index().fillna(method='ffill')
    df.columns = ['sched_group_energy', 'EnergyModel', 'EnergyModel (flat)', 'EnergyModel (flat PDs)']
    
    return df

In [21]:
em.flatten_power_domains().root_pd

PowerDomain(children=[PowerDomain(cpus=[0], idle_states=['WFI', 'cpu-sleep-0', 'cluster-sleep-0']), PowerDomain(cpus=[1], idle_states=['WFI', 'cpu-sleep-0', 'cluster-sleep-0']), PowerDomain(cpus=[2], idle_states=['WFI', 'cpu-sleep-0', 'cluster-sleep-0']), PowerDomain(cpus=[3], idle_states=['WFI', 'cpu-sleep-0', 'cluster-sleep-0'])], idle_states=[])

In [30]:
# plot_columns = ['sched_group_energy', 'EnergyModel', 'EnergyModel (flat)', 'EnergyModel (flat PDs)']
plot_columns = [
    'EnergyModel', 
    'EnergyModel (flat PDs)',
    'EnergyModel (flat)',
    'sched_group_energy'
]

df = get_estimations_df(trace, component=None, column=None)
ILinePlot(df, column=plot_columns, 
          drawstyle='steps-after', title='Energy estimation comparison').view()

""
""


In [27]:
def signal_value_at_time(signal, time):
    return signal.reindex([time], method='ffill').iloc[0]

In [28]:
def examine_sample(trace, t, nrg_model=pixel_energy, column=None, component=None):
    parser = Parser(trace.ftrace)
    # freq = parser.solve('cpu_frequency:frequency')
    idle = parser.solve('cpu_idle:state')
    util = parser.solve('sched_load_avg_cpu:util_avg')
    _inputs = pd.concat([idle, util], axis=1,
                       keys=['idle', 'util'])
    inputs = _inputs.fillna(method='ffill').drop_duplicates()
    df = get_estimations_df(trace, component='idle', column=(2,))

    print "inputs:"
    print signal_value_at_time(inputs, t)
    
    print signal_value_at_time(util_df, t)

    print "\n"
    idle_idxs = [int(i) for i in signal_value_at_time(idle, t)]
    print idle_idxs
    idle_states = [n.idle_state_by_idx(i) for i, n in zip(idle_idxs, nrg_model.cpu_nodes)]
    print idle_states
    print nrg_model.estimate_from_cpu_util([int(u) for u in signal_value_at_time(util, t)],
                                           idle_states=idle_states, combine=False)[2,]

    df = get_estimations_df(trace, nrg_model=nrg_model, column=column, component=component)
    
    print "\n"
    print "sched:"
    print signal_value_at_time(df['sched_group_energy'], t)

    print "\n"
    print "mimiced:"
    print signal_value_at_time(df['EnergyModel'], t)

In [29]:
examine_sample(trace, 3.1, column=(0,), component='active')

inputs:
idle  0     -1.0
      1      2.0
      2      2.0
      3      2.0
util  0    546.0
      1      1.0
      2      1.0
      3      0.0
Name: 3.1, dtype: float64
cpu
0    294.0
1      0.0
2      1.0
3      0.0
Name: 3.1, dtype: float64


[-1, 2, 2, 2]
['cluster-sleep-0', 'cluster-sleep-0', 'cluster-sleep-0', 'cluster-sleep-0']
{'active': 0.6241610738255033, 'idle': 0.0}


sched:
195.0


mimiced:
196.0


In [ ]:
import json
with open('scratchpad/EM_Topo_effect/platform.json') as f:
    platform = json.load(f)

events = ['sched_energy_diff', 'sched_switch', 'sched_load_avg_cpu', 'cpu_idle', 'sg_energy']

trace = Trace(platform, './scratchpad/EM_Topo_effect', events)

In [ ]:
trace.data_dir

In [ ]:
em.cpu_nodes[0].

In [ ]:
[n.idle_state_by_idx(0) for n in em.cpu_nodes]

In [ ]:
# Helper to conver an int into a list of the bits set in it
def to_bits(n):
    ret = []
    for i in range(6):
        if n % 2:
            ret += [i]
        n /= 2
    return tuple(ret)

# Take a DataFrame with columns like ['0x1', '0x2', '0x4', '0x6']
# Return one with columns like       [(0,),  (1,),  (2,),  (1, 2)]
# i.e. convert columns from hex strings representing cpumasks to tuples with logical CPU numbers.
def fix_cols(df):
    df = pd.DataFrame(df)
    df.columns = [to_bits(int(col, 0)) for col in df.columns]
    return df

sched_idle_df = fix_cols(df['idle']).fillna(method='ffill')
sched_active_df = fix_cols(df['active']).fillna(method='ffill')

def examine(test, experiment_idx=0, plot_residency=True, plot_freqs=True):
    ex = test.executor.experiments[experiment_idx]
    
    # Show task residency
    trace = test.get_trace(ex)
    if plot_freqs:
        plot_trace(trace.ftrace)
    
    if plot_freqs and 'cpu_frequency' in trace.available_events:
        p = Parser(trace.ftrace)
        freq_df = p.solve('cpu_frequency:frequency')
        ILinePlot(freq_df, drawstyle='steps-post', title="CPU Frequency").view()
    
    csv_path = os.path.join(ex.out_dir, "samples_Device0.csv")
    if os.path.exists(csv_path):
        # Show recorded energy
        samples_df = pd.read_csv(csv_path, sep=", ", engine='python', index_col='"timestamp ms"')
        samples_df.index.name='time'
        samples_df.index /= 1000
        samples_df = samples_df[['"power mW"']]
        samples_df.columns = ['power']
        ILinePlot(samples_df, column='power', title="Recorded energy").view()

    print "ESTIMATED ENERGY:"
    df = get_estimations_df(trace)

    display(df.corr())
    ILinePlot(df, column=df.columns.tolist(), 
              drawstyle='steps-after', title='Energy estimation comparison').view()
    
df = examine(tests[0], plot_freqs=False, plot_residency=False)

In [ ]:
df.corr()